# 20241007 


새로운 로직을 추가하고 싶어.   wafer반지름 길이(radius)를 이용해서 wafer의 0~50mm(center), 51~100mm(middle), 101~140mm(edge)  3개 영역으로 나눠서 plot을 보고 싶어.   radius는 sqrt((wf_x^2)+(wf_y^2)) 으로 계산할 수 있어. 
ChatGPT

12초 동안 생각

원하시는 기능을 추가해 드리겠습니다. 웨이퍼의 좌표(wf_x, wf_y)를 이용하여 각 데이터 포인트의 반지름(radius)을 계산하고, 이를 기준으로 웨이퍼를 세 개의 영역으로 나누어 플롯을 생성하겠습니다:

센터 (Center): 반지름 0 ~ 50mm (0 ~ 50,000μm)
미들 (Middle): 반지름 51 ~ 100mm (50,001 ~ 100,000μm)
에지 (Edge): 반지름 101 ~ 140mm (100,001 ~ 140,000μm)
구현 방법:
반지름 계산 및 영역 할당:

각 데이터 포인트에 대해 radius = sqrt(wf_x^2 + wf_y^2)를 계산합니다.
계산된 반지름을 기준으로 영역(region)을 할당합니다.
플롯 윈도우에 영역 선택 기능 추가:

PlotWindow 클래스에 영역을 선택할 수 있는 UI 요소(예: QComboBox 또는 QRadioButton)를 추가합니다.
선택된 영역에 해당하는 데이터만 필터링하여 플롯합니다.
플롯 함수 수정:

plot_overlay 함수나 플롯 생성 부분을 수정하여 필터링된 데이터만 사용합니다.
단계별 코드 수정:
1. 반지름 계산 및 영역 할당
PlotWindow 클래스에서 데이터를 로드한 후 반지름을 계산하고 영역을 할당합니다.

```
class PlotWindow(QMainWindow):
    def __init__(self, unique_id, df):
        super().__init__()
        self.setWindowTitle(f"Plot for {unique_id}")
        self.setGeometry(100, 100, 1200, 800)

        # 데이터 필터링
        df_lot = df[df['UNIQUE_ID'] == unique_id]

        # 반지름 계산
        df_lot['radius'] = np.sqrt(df_lot['wf_x']**2 + df_lot['wf_y']**2)

        # 영역 할당
        conditions = [
            (df_lot['radius'] <= 50000),
            (df_lot['radius'] > 50000) & (df_lot['radius'] <= 100000),
            (df_lot['radius'] > 100000) & (df_lot['radius'] <= 140000)
        ]
        choices = ['Center', 'Middle', 'Edge']
        df_lot['region'] = np.select(conditions, choices, default='Outside')

        # 데이터 저장
        self.df_lot = df_lot

        # UI 구성 요소 추가 (아래에서 설명)
        self.init_ui()

``` 



2. 영역 선택 UI 추가
PlotWindow 클래스에 영역을 선택할 수 있는 UI 요소를 추가합니다.







아 근데 문제가 있어.   클릭하면 확대된 창이 떠야하는데,  그게 안돼. 
ChatGPT

15초 동안 생각

죄송하지만 클릭 시 확대된 창이 나타나지 않는 문제가 발생하셨군요. 이 문제를 해결하기 위해 몇 가지 가능한 원인과 해결 방법을 함께 살펴보겠습니다.

문제 원인 분석 및 해결 방법
이벤트 연결 문제:

update_plot() 함수에서 플롯을 갱신할 때 fig.clear()를 사용하여 기존 플롯을 지우고 새로운 플롯을 그립니다. 이 과정에서 이전에 연결된 이벤트 핸들러도 함께 제거됩니다.

해결 방법:

플롯을 업데이트한 후에 클릭 이벤트를 다시 연결해야 합니다.
이벤트 연결 시 이전에 연결된 이벤트를 해제한 후 다시 연결하는 것이 좋습니다.
코드 수정:

python
코드 복사
class PlotWindow(QMainWindow):
    def __init__(self, unique_id, df):
        # ... 기존 코드 ...

        self.cid = None  # 이벤트 연결 ID를 저장할 변수 추가

        # 초기 플롯 생성
        self.update_plot()
python
코드 복사
    def update_plot(self):
        # ... 기존 코드 ...

        # 이전에 연결된 이벤트가 있으면 해제
        if self.cid is not None:
            self.canvas.mpl_disconnect(self.cid)

        # 클릭 이벤트 연결
        self.cid = self.canvas.mpl_connect('button_press_event', self.on_click)
확대된 창에 대한 참조 유지 문제:

Python에서는 객체에 대한 참조를 유지하지 않으면 가비지 컬렉션에 의해 객체가 사라질 수 있습니다. 따라서 확대된 창 객체를 리스트에 저장하여 참조를 유지해야 합니다.

해결 방법:

self.enlarged_plot_windows 리스트에 생성된 확대 창을 추가하여 참조를 유지합니다.
코드 확인:

python
코드 복사
    def on_click(self, event):
        for quiver, title in self.quivers:
            if event.inaxes == quiver.axes and quiver.contains(event)[0]:
                # 선택된 영역에 해당하는 데이터 사용
                selected_region = self.region_combo.currentText()
                data = self.df_lot if selected_region == 'All' else self.df_lot[self.df_lot['region'] == selected_region]

                wf_x = data['wf_x'].tolist()
                wf_y = data['wf_y'].tolist()
                dx = quiver.U
                dy = quiver.V

                enlarged_window = EnlargedPlotWindow(
                    wf_x, wf_y, dx, dy,
                    title=title + f' - {selected_region}',
                    v_lines=self.vertical_lines,
                    h_lines=self.horizontal_lines
                )
                self.enlarged_plot_windows.append(enlarged_window)  # 참조 유지
                enlarged_window.show()
클릭 이벤트 조건 수정:

이벤트가 발생한 축(event.inaxes)과 quiver 객체의 축이 일치하는지 확인해야 합니다. 또한, quiver.contains(event)가 올바르게 동작하도록 수정해야 합니다.

코드 수정:

python
코드 복사
    def on_click(self, event):
        for quiver, title in self.quivers:
            if event.inaxes == quiver.axes and quiver.contains(event)[0]:
                # ... 기존 코드 ...
플롯 업데이트 시 객체 참조 문제:

update_plot() 함수에서 self.quivers를 업데이트하지만, 확대 창에서는 이전 quiver 객체를 참조할 수 있습니다.

해결 방법:

self.quivers를 업데이트할 때마다 기존 참조를 갱신합니다.
코드 확인:

python
코드 복사
    def update_plot(self):
        # ... 기존 코드 ...

        self.quivers = []

        # 플롯 생성 루프
        for config in plot_configs:
            quiver = plot_overlay(config['ax'], wf_x, wf_y, config['dx'], config['dy'],
                                  self.vertical_lines, self.horizontal_lines,
                                  title=config['title'])
            self.quivers.append((quiver, config['title']))
전체 수정된 코드
아래는 위에서 언급한 수정 사항을 반영한 PlotWindow 클래스의 전체 코드입니다.

python
코드 복사
class PlotWindow(QMainWindow):
    def __init__(self, unique_id, df):
        super().__init__()
        self.setWindowTitle(f"Plot for {unique_id}")
        self.setGeometry(100, 100, 1200, 800)

        # 데이터 필터링
        df_lot = df[df['UNIQUE_ID'] == unique_id]

        # 반지름 계산
        df_lot['radius'] = np.sqrt(df_lot['wf_x']**2 + df_lot['wf_y']**2)

        # 영역 할당
        conditions = [
            (df_lot['radius'] <= 50000),
            (df_lot['radius'] > 50000) & (df_lot['radius'] <= 100000),
            (df_lot['radius'] > 100000) & (df_lot['radius'] <= 140000)
        ]
        choices = ['Center', 'Middle', 'Edge']
        df_lot['region'] = np.select(conditions, choices, default='Outside')

        # 데이터 저장
        self.df_lot = df_lot

        # 확대된 창들을 저장할 리스트
        self.enlarged_plot_windows = []

        # 이벤트 연결 ID를 저장할 변수
        self.cid = None

        # UI 구성 요소 추가
        self.init_ui()

    def init_ui(self):
        # 메인 위젯과 레이아웃 생성
        main_widget = QWidget()
        main_layout = QVBoxLayout()

        # 영역 선택 콤보박스 추가
        region_label = QLabel("Select Region:")
        self.region_combo = QComboBox()
        self.region_combo.addItems(['All', 'Center', 'Middle', 'Edge'])
        self.region_combo.setCurrentIndex(0)  # 기본값을 'All'로 설정
        self.region_combo.currentIndexChanged.connect(self.update_plot)

        # 플롯 캔버스와 툴바 생성
        self.fig, self.axes = plt.subplots(4, 3, figsize=(12, 8))
        self.canvas = FigureCanvas(self.fig)
        self.toolbar = NavigationToolbar(self.canvas, self)

        # 레이아웃에 위젯 추가
        region_layout = QHBoxLayout()
        region_layout.addWidget(region_label)
        region_layout.addWidget(self.region_combo)

        main_layout.addLayout(region_layout)
        main_layout.addWidget(self.toolbar)
        main_layout.addWidget(self.canvas)

        main_widget.setLayout(main_layout)
        self.setCentralWidget(main_widget)

        # 초기 플롯 생성
        self.update_plot()

    def update_plot(self):
        # 선택된 영역 가져오기
        selected_region = self.region_combo.currentText()

        # 데이터 필터링
        if selected_region == 'All':
            data = self.df_lot
        else:
            data = self.df_lot[self.df_lot['region'] == selected_region]

        # 데이터 추출
        wf_x = data['wf_x'].tolist()
        wf_y = data['wf_y'].tolist()

        # 필요한 변수들도 동일하게 필터링
        variables = ['X_reg', 'Y_reg', 'pred_x', 'pred_y', 'residual_x', 'residual_y',
                     'psm_fit_x', 'psm_fit_y', 'residual_x_depsm', 'residual_y_depsm',
                     'cpe19p_pred_x', 'cpe19p_pred_y', 'cpe19p_resi_x', 'cpe19p_resi_y',
                     'ideal_psm_x', 'ideal_psm_y', 'delta_psm_x', 'delta_psm_y']

        data_vars = {var: data[var].tolist() for var in variables}

        # 기존 플롯 초기화
        self.fig.clear()
        axes = self.fig.subplots(4, 3)
        self.fig.suptitle(f'Visualizations for Lot {self.df_lot["UNIQUE_ID"].iloc[0]} - {selected_region}', fontsize=16)

        self.quivers = []

        # 스텝 피치 및 맵 시프트 추출 (이전 코드에서 가져옴)
        step_pitch_x = self.df_lot['STEP_PITCH_X'].iloc[0]
        step_pitch_y = self.df_lot['STEP_PITCH_Y'].iloc[0]
        map_shift_x = self.df_lot['MAP_SHIFT_X'].iloc[0]
        map_shift_y = self.df_lot['MAP_SHIFT_Y'].iloc[0]
        start_left = -(step_pitch_x)/2 + map_shift_x
        start_bottom = -(step_pitch_y)/2 + map_shift_y
        max_die_x = int(self.df_lot['DieX'].max())
        min_die_x = int(self.df_lot['DieX'].min())
        max_die_y = int(self.df_lot['DieY'].max())
        min_die_y = int(self.df_lot['DieY'].min())

        self.vertical_lines = calculate_lines(start_left, step_pitch_x, max_die_x, min_die_x)
        self.horizontal_lines = calculate_lines(start_bottom, step_pitch_y, max_die_y, min_die_y)

        # 플롯 설정 정보를 리스트로 정의
        plot_configs = [
            {'ax': axes[0, 0], 'dx': data_vars['X_reg'], 'dy': data_vars['Y_reg'], 'title': 'Raw(X_reg,Y_reg)'},
            {'ax': axes[0, 1], 'dx': data_vars['pred_x'], 'dy': data_vars['pred_y'], 'title': 'OSR_Fitting(WK,RK)'},
            {'ax': axes[0, 2], 'dx': data_vars['residual_x'], 'dy': data_vars['residual_y'], 'title': 'Residual'},
            {'ax': axes[1, 0], 'dx': data_vars['psm_fit_x'], 'dy': data_vars['psm_fit_y'], 'title': 'PSM Input'},
            {'ax': axes[1, 1], 'dx': data_vars['residual_x_depsm'], 'dy': data_vars['residual_y_depsm'], 'title': 'Residual(Remove_PSM)'},
            {'ax': axes[1, 2], 'dx': data_vars['cpe19p_pred_x'], 'dy': data_vars['cpe19p_pred_y'], 'title': 'CPE 19para Fitting'},
            {'ax': axes[2, 0], 'dx': data_vars['cpe19p_resi_x'], 'dy': data_vars['cpe19p_resi_y'], 'title': 'CPE 19para Residual'},
            {'ax': axes[2, 1], 'dx': data_vars['ideal_psm_x'], 'dy': data_vars['ideal_psm_y'], 'title': 'Ideal PSM'},
            {'ax': axes[2, 2], 'dx': data_vars['delta_psm_x'], 'dy': data_vars['delta_psm_y'], 'title': 'Delta PSM'},
            {'ax': axes[3, 0], 'dx': data_vars['delta_psm_x'], 'dy': [0]*len(data_vars['delta_psm_y']), 'title': 'Delta PSM X'},
            {'ax': axes[3, 1], 'dx': [0]*len(data_vars['delta_psm_x']), 'dy': data_vars['delta_psm_y'], 'title': 'Delta PSM Y'},
            # 필요에 따라 더 추가 가능
        ]

        for config in plot_configs:
            quiver = plot_overlay(config['ax'], wf_x, wf_y, config['dx'], config['dy'],
                                  self.vertical_lines, self.horizontal_lines,
                                  title=config['title'])
            self.quivers.append((quiver, config['title']))

        # 사용되지 않는 서브플롯 숨기기 (예: axes[3,2])
        axes[3, 2].axis('off')

        # 캔버스 갱신
        self.canvas.draw()

        # 이전에 연결된 이벤트가 있으면 해제
        if self.cid is not None:
            self.canvas.mpl_disconnect(self.cid)

        # 클릭 이벤트 연결
        self.cid = self.canvas.mpl_connect('button_press_event', self.on_click)

    def on_click(self, event):
        for quiver, title in self.quivers:
            if event.inaxes == quiver.axes and quiver.contains(event)[0]:
                # 선택된 영역에 해당하는 데이터 사용
                selected_region = self.region_combo.currentText()
                data = self.df_lot if selected_region == 'All' else self.df_lot[self.df_lot['region'] == selected_region]

                wf_x = data['wf_x'].tolist()
                wf_y = data['wf_y'].tolist()
                dx = quiver.U
                dy = quiver.V

                enlarged_window = EnlargedPlotWindow(
                    wf_x, wf_y, dx, dy,
                    title=title + f' - {selected_region}',
                    v_lines=self.vertical_lines,
                    h_lines=self.horizontal_lines
                )
                self.enlarged_plot_windows.append(enlarged_window)
                enlarged_window.show()
추가 확인 사항
에러 메시지 확인:

프로그램을 실행할 때 터미널이나 콘솔에 에러 메시지가 출력되는지 확인해 주세요. 에러 메시지는 문제를 해결하는 데 큰 도움이 됩니다.

라이브러리 버전 확인:

Matplotlib 및 PyQt5의 버전 차이로 인해 이벤트 연결이 정상적으로 동작하지 않을 수 있습니다. 최신 버전으로 업데이트하거나 호환되는 버전을 사용해 보세요.

환경 테스트:

가능하다면 간단한 예제 코드를 작성하여 클릭 이벤트와 확대 창이 정상적으로 동작하는지 테스트해 보세요. 이를 통해 문제의 원인이 코드의 다른 부분인지 환경적인 문제인지 파악할 수 있습니다.

결론
위의 수정 사항을 적용하시면 클릭 시 확대된 창이 정상적으로 표시될 것입니다. 문제가 지속되거나 추가로 도움이 필요하시면 언제든지 말씀해 주세요. 함께 해결해 드리겠습니다!

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.patches import Circle
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout,
    QPushButton, QLabel, QFileDialog, QLineEdit, QListWidget, QMessageBox, QComboBox, QHBoxLayout
)
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.backends.backend_qt5agg import NavigationToolbar2QT as NavigationToolbar

# 상수 및 설정 값 관리
class Config:
    WAFER_RADIUS = 150000
    SCALE_FACTOR = 1e-7
    SCALE_BAR_LENGTH = 30000
    SCALE_BAR_POSITION = 'lower center'
    SCALE_BAR_LENGTH_PIXELS = 30000
    SCALE_BAR_SIZE_VERTICAL = 500
    TEXT_POSITION_Y = -170000  # 텍스트 추가 위치

# 가로선과 세로선을 계산하는 함수 통합
def calculate_lines(center, pitch, max_die, min_die):
    lines = []
    current = center
    for _ in range(max_die + 2):
        lines.append(current)
        current += pitch
    current = center
    for _ in range(abs(min_die) + 1):
        current -= pitch
        lines.append(current)
    return lines

# 통계 계산 함수 (ABS(Mean) + 3sigma)
def calculate_statistics(values):
    mean_val = np.mean(values)
    sigma_val = np.std(values)
    m3s_val = np.abs(mean_val) + 3 * sigma_val
    m3s_nm = m3s_val * 1e3  # nm 단위로 변환
    return m3s_nm

# Overlay를 플롯하는 함수
def plot_overlay(ax, x, y, dx, dy, v_lines, h_lines, wafer_radius=Config.WAFER_RADIUS,
                 title='Wafer Vector Map', scale_factor=Config.SCALE_FACTOR,
                 show_statistics=True):
    quiver = ax.quiver(x, y, dx, dy, angles='xy', scale_units='xy', scale=scale_factor,
                       color='blue', label='Overlay Vectors',
                       width=0.0015, headwidth=3, headlength=3)

    ax.axvline(0, color='red', linewidth=1.0, label='Central X')
    ax.axhline(0, color='red', linewidth=1.0, label='Central Y')

    for vline in v_lines:
        ax.axvline(vline, color='black', linestyle='--', linewidth=0.8)
    for hline in h_lines:
        ax.axhline(hline, color='black', linestyle='--', linewidth=0.8)

    wafer_circle = Circle((0, 0), wafer_radius, color='green', fill=False,
                          linestyle='-', linewidth=2, label='Wafer Boundary')
    ax.add_patch(wafer_circle)

    scale_bar_length = Config.SCALE_BAR_LENGTH * scale_factor
    scale_bar_label = f'{scale_bar_length * 1e3:.1f}nm'

    fontprops = fm.FontProperties(size=10)
    scalebar = AnchoredSizeBar(ax.transData, Config.SCALE_BAR_LENGTH_PIXELS, scale_bar_label,
                               Config.SCALE_BAR_POSITION, pad=0.1, color='black',
                               frameon=False, size_vertical=Config.SCALE_BAR_SIZE_VERTICAL,
                               fontproperties=fontprops)
    ax.add_artist(scalebar)

    # 통계치 계산 및 텍스트 추가
    if show_statistics:
        mean_plus_3sigma_x = calculate_statistics(dx)
        mean_plus_3sigma_y = calculate_statistics(dy)
        ax.text(0, Config.TEXT_POSITION_Y,
                f'|m|+3s X: {mean_plus_3sigma_x:.2f} nm\n|m|+3s Y: {mean_plus_3sigma_y:.2f} nm',
                fontsize=10, color='red', ha='center')

    ax.set_xlabel('Wafer X Coordinate (wf_x)')
    ax.set_ylabel('Wafer Y Coordinate (wf_y)')
    ax.set_title(title)
    ax.axis('equal')
    ax.grid(False)

    return quiver

# 클릭한 플롯을 확대해서 보여주는 클래스
class EnlargedPlotWindow(QMainWindow):
    def __init__(self, x, y, u, v, title, v_lines, h_lines,
                 wafer_radius=Config.WAFER_RADIUS, scale_factor=Config.SCALE_FACTOR):
        super().__init__()
        self.setWindowTitle(title)
        self.setGeometry(100, 100, 800, 800)

        # 확대된 플롯을 생성
        fig, ax = plt.subplots(figsize=(8, 8))

        # plot_overlay 함수 사용
        plot_overlay(ax, x, y, u, v, v_lines, h_lines, wafer_radius, title, scale_factor)

        # Matplotlib FigureCanvas 생성
        self.canvas = FigureCanvas(fig)

        # Navigation Toolbar 추가
        self.toolbar = NavigationToolbar(self.canvas, self)

        layout = QVBoxLayout()
        layout.addWidget(self.toolbar)
        layout.addWidget(self.canvas)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

# 플롯을 보여주는 클래스
class PlotWindow(QMainWindow):
    def __init__(self, unique_id, df):
        super().__init__()
        self.setWindowTitle(f"Plot for {unique_id}")
        self.setGeometry(100, 100, 1200, 800)

        # 데이터 필터링
        df_lot = df[df['UNIQUE_ID'] == unique_id]

        # 반지름 계산
        df_lot['radius'] = np.sqrt(df_lot['wf_x']**2 + df_lot['wf_y']**2)

        # 영역 할당
        conditions = [
            (df_lot['radius'] <= 50000),
            (df_lot['radius'] > 50000) & (df_lot['radius'] <= 100000),
            (df_lot['radius'] > 100000) & (df_lot['radius'] <= 150000)
        ]
        choices = ['Center', 'Middle', 'Edge']
        df_lot['region'] = np.select(conditions, choices, default='Outside')

        # 데이터 저장
        self.df_lot = df_lot

        # 확대된 창들을 저장할 리스트
        self.enlarged_plot_windows = []

        # 이벤트 연결 ID를 저장할 변수
        self.cid = None

        # UI 구성 요소 추가
        self.init_ui()

    def init_ui(self):
        # 메인 위젯과 레이아웃 생성
        main_widget = QWidget()
        main_layout = QVBoxLayout()

        # 영역 선택 콤보박스 추가
        region_label = QLabel("Select Region:")
        self.region_combo = QComboBox()
        self.region_combo.addItems(['All', 'Center', 'Middle', 'Edge'])
        self.region_combo.setCurrentIndex(0)  # 기본값을 'All'로 설정
        self.region_combo.currentIndexChanged.connect(self.update_plot)

        # 플롯 캔버스와 툴바 생성
        self.fig, self.axes = plt.subplots(4, 3, figsize=(12, 8))
        self.canvas = FigureCanvas(self.fig)
        self.toolbar = NavigationToolbar(self.canvas, self)

        # 레이아웃에 위젯 추가
        region_layout = QHBoxLayout()
        region_layout.addWidget(region_label)
        region_layout.addWidget(self.region_combo)

        main_layout.addLayout(region_layout)
        main_layout.addWidget(self.toolbar)
        main_layout.addWidget(self.canvas)

        main_widget.setLayout(main_layout)
        self.setCentralWidget(main_widget)

        # 초기 플롯 생성
        self.update_plot()

    def update_plot(self):
        # 선택된 영역 가져오기
        selected_region = self.region_combo.currentText()

        # 데이터 필터링
        if selected_region == 'All':
            data = self.df_lot
        else:
            data = self.df_lot[self.df_lot['region'] == selected_region]

        # 데이터 추출
        wf_x = data['wf_x'].tolist()
        wf_y = data['wf_y'].tolist()

        # 필요한 변수들도 동일하게 필터링
        variables = ['X_reg', 'Y_reg', 'pred_x', 'pred_y', 'residual_x', 'residual_y',
                     'psm_fit_x', 'psm_fit_y', 'residual_x_depsm', 'residual_y_depsm',
                     'cpe19p_pred_x', 'cpe19p_pred_y', 'cpe19p_resi_x', 'cpe19p_resi_y',
                     'ideal_psm_x', 'ideal_psm_y', 'delta_psm_x', 'delta_psm_y']

        data_vars = {var: data[var].tolist() for var in variables}

        # 기존 플롯 초기화
        self.fig.clear()
        axes = self.fig.subplots(4, 3)
        self.fig.suptitle(f'Visualizations for Lot {self.df_lot["UNIQUE_ID"].iloc[0]} - {selected_region}', fontsize=16)

        self.quivers = []

        # 스텝 피치 및 맵 시프트 추출 (이전 코드에서 가져옴)
        step_pitch_x = self.df_lot['STEP_PITCH_X'].iloc[0]
        step_pitch_y = self.df_lot['STEP_PITCH_Y'].iloc[0]
        map_shift_x = self.df_lot['MAP_SHIFT_X'].iloc[0]
        map_shift_y = self.df_lot['MAP_SHIFT_Y'].iloc[0]
        start_left = -(step_pitch_x)/2 + map_shift_x
        start_bottom = -(step_pitch_y)/2 + map_shift_y
        max_die_x = int(self.df_lot['DieX'].max())
        min_die_x = int(self.df_lot['DieX'].min())
        max_die_y = int(self.df_lot['DieY'].max())
        min_die_y = int(self.df_lot['DieY'].min())

        self.vertical_lines = calculate_lines(start_left, step_pitch_x, max_die_x, min_die_x)
        self.horizontal_lines = calculate_lines(start_bottom, step_pitch_y, max_die_y, min_die_y)

        # 플롯 설정 정보를 리스트로 정의
        plot_configs = [
            {'ax': axes[0, 0], 'dx': data_vars['X_reg'], 'dy': data_vars['Y_reg'], 'title': 'Raw(X_reg,Y_reg)'},
            {'ax': axes[0, 1], 'dx': data_vars['pred_x'], 'dy': data_vars['pred_y'], 'title': 'OSR_Fitting(WK,RK)'},
            {'ax': axes[0, 2], 'dx': data_vars['residual_x'], 'dy': data_vars['residual_y'], 'title': 'Residual'},
            {'ax': axes[1, 0], 'dx': data_vars['psm_fit_x'], 'dy': data_vars['psm_fit_y'], 'title': 'PSM Input'},
            {'ax': axes[1, 1], 'dx': data_vars['residual_x_depsm'], 'dy': data_vars['residual_y_depsm'], 'title': 'Residual(Remove_PSM)'},
            {'ax': axes[1, 2], 'dx': data_vars['cpe19p_pred_x'], 'dy': data_vars['cpe19p_pred_y'], 'title': 'CPE 19para Fitting'},
            {'ax': axes[2, 0], 'dx': data_vars['cpe19p_resi_x'], 'dy': data_vars['cpe19p_resi_y'], 'title': 'CPE 19para Residual'},
            {'ax': axes[2, 1], 'dx': data_vars['ideal_psm_x'], 'dy': data_vars['ideal_psm_y'], 'title': 'Ideal PSM'},
            {'ax': axes[2, 2], 'dx': data_vars['delta_psm_x'], 'dy': data_vars['delta_psm_y'], 'title': 'Delta PSM'},
            {'ax': axes[3, 0], 'dx': data_vars['delta_psm_x'], 'dy': [0]*len(data_vars['delta_psm_y']), 'title': 'Delta PSM X'},
            {'ax': axes[3, 1], 'dx': [0]*len(data_vars['delta_psm_x']), 'dy': data_vars['delta_psm_y'], 'title': 'Delta PSM Y'},
            # 필요에 따라 더 추가 가능
        ]

        for config in plot_configs:
            quiver = plot_overlay(config['ax'], wf_x, wf_y, config['dx'], config['dy'],
                                  self.vertical_lines, self.horizontal_lines,
                                  title=config['title'])
            self.quivers.append((quiver, config['title']))

        # 사용되지 않는 서브플롯 숨기기 (예: axes[3,2])
        axes[3, 2].axis('off')

        # 캔버스 갱신
        self.canvas.draw()

        # 이전에 연결된 이벤트가 있으면 해제
        if self.cid is not None:
            self.canvas.mpl_disconnect(self.cid)

        # 클릭 이벤트 연결
        self.cid = self.canvas.mpl_connect('button_press_event', self.on_click)

    def on_click(self, event):
        for quiver, title in self.quivers:
            if event.inaxes == quiver.axes and quiver.contains(event)[0]:
                # 선택된 영역에 해당하는 데이터 사용
                selected_region = self.region_combo.currentText()
                data = self.df_lot if selected_region == 'All' else self.df_lot[self.df_lot['region'] == selected_region]

                wf_x = data['wf_x'].tolist()
                wf_y = data['wf_y'].tolist()
                dx = quiver.U
                dy = quiver.V

                enlarged_window = EnlargedPlotWindow(
                    wf_x, wf_y, dx, dy,
                    title=title + f' - {selected_region}',
                    v_lines=self.vertical_lines,
                    h_lines=self.horizontal_lines
                )
                self.enlarged_plot_windows.append(enlarged_window)
                enlarged_window.show()


# 메인 윈도우
class MainWindow(QMainWindow):
    def __init__(self, df):
        super().__init__()
        self.setWindowTitle("Unique ID Plot Viewer")
        self.setGeometry(100, 100, 400, 400)

        layout = QVBoxLayout()

        # 라벨 추가
        label = QLabel("Select a unique_id to view the plot:")
        layout.addWidget(label)

        # 검색 입력창 추가
        self.search_box = QLineEdit()
        self.search_box.setPlaceholderText("Search unique_id")
        layout.addWidget(self.search_box)

        # 리스트 위젯 추가
        self.list_widget = QListWidget()
        layout.addWidget(self.list_widget)

        # DataFrame에서 unique_id 목록 추출
        self.unique_ids = sorted(df['UNIQUE_ID'].unique())
        self.list_widget.addItems(self.unique_ids)

        # 검색 기능 연결
        self.search_box.textChanged.connect(self.filter_unique_ids)

        # 버튼 추가
        button = QPushButton("Show Plot")
        button.clicked.connect(self.show_plot)
        layout.addWidget(button)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        # 데이터 저장
        self.df = df

    def filter_unique_ids(self, text):
        self.list_widget.clear()
        filtered_ids = [uid for uid in self.unique_ids if text.lower() in uid.lower()]
        self.list_widget.addItems(filtered_ids)

    def show_plot(self):
        selected_items = self.list_widget.selectedItems()
        if selected_items:
            unique_id = selected_items[0].text()
            self.plot_window = PlotWindow(unique_id, self.df)
            self.plot_window.show()
        else:
            QMessageBox.warning(self, "No Selection", "Please select a unique_id from the list.")

# PyQt 애플리케이션 실행
def main():
    app = QApplication([])
    file_dialog = QFileDialog()
    file_path, _ = file_dialog.getOpenFileName(None, "Select CSV File", "", "CSV Files (*.csv)")
    if not file_path:
        return
    try:
        # CSV 데이터 로드
        df = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error loading file: {e}")
        return

    main_window = MainWindow(df)
    main_window.show()
    app.exec_()

if __name__ == "__main__":
    main()




C:\Users\oth08\AppData\Local\Temp\ipykernel_4224\3113209575.py:130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lot['radius'] = np.sqrt(df_lot['wf_x']**2 + df_lot['wf_y']**2)
C:\Users\oth08\AppData\Local\Temp\ipykernel_4224\3113209575.py:139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lot['region'] = np.select(conditions, choices, default='Outside')


: 